**Load data**

Load from csv into pandas. Each sequence, one by one.

In [27]:
from util.data import preprocess, load_data

assets = load_data('data')
asset = preprocess(assets[0])

[[0.76190476]
 [0.80952381]
 [0.85714286]
 [0.9047619 ]
 [0.95238095]
 [1.        ]]


**Model**

Initialize Model

In [ ]:
from tensorflow import keras



**Training**

Run training.